# VRTs: Stack Sentinel-1 RTC products into time series data

This jupyter notebook describes how to stack Sentinel-1 data into time series with Virtual Raster Tables

In [ ]:
# Import python modules
import os, sys
import datetime
import glob
import subprocess as sp
import tempfile
import gdal

Set the root and RTC directories for gdalbuildvrt

In [ ]:
# Set DryRun to True for a list of the gdalbuildvrt commands only without making the call
DryRun=False

# Give the root directory where the subdirectories containing the RTC files are stored
rootdir='../Retrieved Data/SAR_CH3/wa_v2/BIOsS1'

# Give list of RTC directories relative to the rootdir
rtcdirs=['S32631X398020Y1315440sS1_A_vh_mtfil','S32631X398020Y1315440sS1_A_vv_mtfil']

# Give a target extent in projection coordinates
targetextent=None

# Give targetextent as "xmin ymin xmax ymax"
# targetextent="398020.000 1315440.000 482880.000 1390960.000 "

Define a function to find all RTC files in a directory

In [ ]:
def make_timeseries_vrt(rootdir,subdir,targetextent=None,prefix='',DryRun=False):
    '''
    Make a vrt stack from time series data stored in a subdirectory 

    Author: Josef Kellndorfer (2018)

    INPUT:
    rootdir        Path to the root directory where the subdirectory is stored
    subdir         Name of the subdirectory containing the time series data as geotiffs (.tif ending)
    targetextent   None or string as "xmin ymin xmax ymax" in projection units
    prefix         Optional prefix for the vrt filename. Default is subdirectory name with .vrt ending
    DryRun         prints only the gdalbuild command with out executing it

    

    Will generate .vrt time series stacks with bands named by dates. 
    '''

    # Get current vorking directory and change to the root directory
    cwd=os.getcwd()
    os.chdir(rootdir)

    imagelist = [x for x in glob.glob(os.path.join(subdir,'*.tif'))]
    # Get index for a valid date in the '_' separated list
    first=imagelist[0].split('_')
    for i in range(len(first)):
        try:
            datetime.datetime.strptime(first[i],'%Y%m%d')
            idx = i
        except ValueError:
            next
    # Now get the dates for all the files
    dates = [x.split('_')[idx] for x in imagelist]
    # Sort by dates and resort the imagelist accordingly
    datessorted = dates.copy()
    datessorted.sort()
    indices = [dates.index(x) for x in datessorted]
    imagelistsorted = [imagelist[x] for x in indices]

    # Get all the distict paths
    paths=[x.split('_')[idx+1] for x in imagelistsorted]
    distinct_paths=set(paths)

    # We make a directory with vrtnames and have filelists as the entry
    vrtnames={}

    # Make a vrtname for the entire stack. We use the subdir name as the root for the vrt file:
    vrtname = prefix + subdir + '.vrt'
    vrtnames[vrtname] = {'filenames':imagelist, 'dates':dates}

    # Now make vrtnames and corresponding lists for files and dates by path:
    for p in distinct_paths:
        # Get the subset of the imagelist that corresponds to this path:
        imagelist_subset=[x for x in imagelistsorted if x.find('_'+p+'_')>-1]
        dates_subset=[x.split('_')[idx] for x in imagelist_subset]
        # VRT name is constructed from the first filename in the rtc list without the 
        vrtname=prefix+subdir+'_'+p+'.vrt'
        vrtnames[vrtname]={'filenames':imagelist_subset, 'dates':dates_subset}

    # Now build the vrts:
    if targetextent==None:
        te=''
    else:
        te='-te '+ targetextent

    for i in vrtnames:
        tfile=tempfile.NamedTemporaryFile(mode='w+',delete=False)
        for j in vrtnames[i]['filenames']:
            tfile.write(j+'\n')
        tfile.close()
        cmd='gdalbuildvrt -separate -allow_projection_difference {} -input_file_list {} -overwrite {}'.format(te,tfile.name,i)
        if not DryRun:
            sp.call(cmd.split())
            # Write the dates files
            datefilename=i.replace('.vrt','.dates')
            with open(datefilename,"w+") as f:
                for j in vrtnames[i]['dates']:
                    f.write(j+'\n')
            # Set the band names as dates
            img_handle=gdal.Open(i)
            for j in range(img_handle.RasterCount):
                x=vrtnames[i]['dates'][j]
                # Reformat YYYYMMDD to YYYY-MM-DD
                date=x[:4]+'-'+x[4:6]+'-'+x[6:8]
                # Set the Metadata and Description to the date
                band=img_handle.GetRasterBand(j+1)
                band.SetMetadataItem("Date",date)
                band.SetDescription(date)
            del img_handle
        else:
            print('DryRun:',end=' ')
        print(cmd)

        os.remove(tfile.name)
    # Switch back to the original working directory
    os.chdir(cwd)

In [ ]:
for i in rtcdirs:
    make_timeseries_vrt(rootdir,i,prefix='',DryRun=False)